Notebook pour m'aider à filtrer les features, leur représentativité etc et juger si elles sont catégorielels ou non

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
# !pip install openml
import openml
# !pip install cardinal
# !pip install scikit-learn==0.20.4
# !pip install sklearn.cluster._k_means_fast
from cardinal.uncertainty import MarginSampler
from cardinal.random import RandomSampler
from cardinal.zhdanov2019 import TwoStepKMeansSampler
from cardinal.plotting import plot_confidence_interval
import tqdm


import os
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import clone_model

In [4]:
np.random.seed(123)

# Détermination manuelle des features catégorielles OpenML

In [5]:
dataset_ids = [1471, 1502, 40922, 41162, 43551, 1461, 1590, 41138, 42395, 42803, 43439]  

In [148]:
dataset = openml.datasets.get_dataset(1461)
X, y, cat_indicator, names = dataset.get_data(dataset_format='dataframe', target=dataset.default_target_attribute)  #dataframe / array
cat_indicator = np.asarray(cat_indicator)

start_index = 0

for feat_name, cat_bool in zip(X.columns[start_index:], cat_indicator[start_index:]):
    if len(np.unique(X[feat_name])) <500:
        print(cat_bool, feat_name,  "       ",len(np.unique(X[feat_name])), np.unique(X[feat_name]) )#   np.unique(X[feat_name]) '        SAMPLE:',X[feat_name][:1])
        print()


X

#features incertaines: 

# idx feat problématiques/inanalysables pour le dataset de voitures :  28, 30,32, 50

False V1         77 [18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89
 90 92 93 94 95]

True V2         12 ['admin.' 'blue-collar' 'entrepreneur' 'housemaid' 'management' 'retired'
 'self-employed' 'services' 'student' 'technician' 'unemployed' 'unknown']

True V3         3 ['divorced' 'married' 'single']

True V4         4 ['primary' 'secondary' 'tertiary' 'unknown']

True V5         2 ['no' 'yes']

True V7         2 ['no' 'yes']

True V8         2 ['no' 'yes']

True V9         3 ['cellular' 'telephone' 'unknown']

False V10         31 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]

True V11         12 ['apr' 'aug' 'dec' 'feb' 'jan' 'jul' 'jun' 'mar' 'may' 'nov' 'oct' 'sep']

False V13         48 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 2

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0.0,unknown
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0.0,unknown
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0.0,unknown
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0.0,unknown
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0.0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825.0,no,no,cellular,17,nov,977.0,3,-1.0,0.0,unknown
45207,71,retired,divorced,primary,no,1729.0,no,no,cellular,17,nov,456.0,2,-1.0,0.0,unknown
45208,72,retired,married,secondary,no,5715.0,no,no,cellular,17,nov,1127.0,5,184.0,3.0,success
45209,57,blue-collar,married,secondary,no,668.0,no,no,telephone,17,nov,508.0,4,-1.0,0.0,unknown


### Examen features incertaines

In [149]:
#features incertaines: education-num, hours-per-week
feat = "V15"
print(len(np.unique(X[feat])))
print(np.unique(X[feat]))

41
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  32.  35.  37.  38.  40.  41.  51.  55.  58. 275.]


### 43439

Drop: PatientId, AppointmentId

Cat: Gender, ScheduledDay, AppointmentDay Neighbourhood, Scholarship, Hipertension, Diabetes, Alcoholism, Handcap, SMS_received
Numeric: Age


### 43551

Add cat : is_male, logical_score, verbal_score

### 1461

Add cat : V1, V13, V15  

### 1590

Add cat : education-num, hours-per-week

### 41138

Suspicieux catégorie ? "eg_000" comme nombreux autres